# Python Não Tão Básico

Aqui mostraremos alguns recursos mais avançados do Python que são úteis para processar e analisar dados.

## Ordenação

Todas as listas em Python tem um método `sort` que a ordena. Se você não quiser alterar a sua lista, você pode usar a função `sorted`, que retorna uma lista nova:

In [2]:
x = [4,1,2,3]
y = sorted(x) 
print(x, '->', y)
x.sort() 
print(x)

[4, 1, 2, 3] -> [1, 2, 3, 4]
[1, 2, 3, 4]


Por padrão, os métodos `sort` e `sorted` ordenam uma lista do menor elemento para o maior a partir de comparações ingênuas dos elementos entre si.

Se você quiser que os elementos sejam ordenados do maior para o menor, você pode especificar o parâmetro `reverse = True`. E em vez de comparar os próprios elementos, você pode comparar os resultados de uma função que você especifica a partir do parâmetro `key`:

In [3]:
# ordena a lista pelo valor absoluto dos elementos, do maior para o menor
x = sorted([-4,1,-2,3], key=abs, reverse=True) 
print(x)

[-4, 3, -2, 1]


Outro exemplo, para ordenar listas de tuplas:

In [4]:
word_counts = [('suppose', 1), ('rose', 3), ('course', 2)]

# ordena as palavras e contagens da maior para a menor
wc = sorted(word_counts,
            key=lambda wcs: wcs[1],
            reverse=True)

print(wc)

[('rose', 3), ('course', 2), ('suppose', 1)]


## Formação de listas

Frequentemente, você quer transformar uma lista em outra lista, escolhendo apenas determinados elementos, ou modificando elementos, ou ambos. A maneira pythônica de fazer isso é através de operadores de formação de listas (*list comprehensions*):

In [5]:
even_numbers = [x for x in range(5) if x % 2 == 0]
print(even_numbers)

squares = [x * x for x in range(5)] 
print(squares)

even_squares = [x * x for x in even_numbers]
print(even_squares)

[0, 2, 4]
[0, 1, 4, 9, 16]
[0, 4, 16]


Você pode também transformar listas em dicionários ou conjuntos:

In [6]:
square_dict = { x : x * x for x in range(5) } 
print(square_dict)

square_set = { x * x for x in [1, -1] }
print(square_set)

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16}
{1}


Se você não precisa dos valores da lista, pode-se usar o *underscore* `_` como a variável:

In [7]:
zeroes = [0 for _ in even_numbers] #mesmo tamanho de even_numbers
print(zeroes)

[0, 0, 0]


A formação de uma lista pode incluir múltiplos `for`s:

In [8]:
pairs = [(x, y)
         for x in range(3)
         for y in range(3)] 
print(pairs)

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]


Os `for`s posteriores podem usar os resultados dos `for`s anteriores:

In [9]:
#apenas pares em que x < y:
increasing_pairs = [(x, y)
                    for x in range(3)
                    for y in range(x + 1, 3)]
print(increasing_pairs)

[(0, 1), (0, 2), (1, 2)]


Usaremos muito operações de formação de listas ao longo do curso.

## Geradores e Iteradores

Um problema com as listas é que elas podem facilmente crescer muito. O comando `range(1000000)` cria uma lista real de 1 milhão de elementos. Se você só precisa de lidar com um elemento de cada vez, isso pode ser uma enorme fonte de ineficiência (ou falta de memória). Ou então, se você potencialmente só precisa dos primeiros valores da lista, então uma lista muito grande é um enorme desperdício.

Um gerador (ou *generator*) é algo que você pode iterar sobre ( geralmente usando `for`), mas cujos valores são produzidos apenas conforme necessário (preguiçosamente).

Uma maneira de criar geradores é com funções e o operador de `yield`:

In [10]:
def lazy_range(n):
    """a lazy version of range"""
    i = 0
    while i < n:
        yield i
        i += 1

O loop seguinte vai consumir os valores lançados (*yielded*) pela função `lazy_range` um de cada vez até que não haja mais elementos para serem lançados:

In [11]:
def do_something_with(value):
    print('what should I do with', value, '?')

for i in lazy_range(10):
    do_something_with(i)

what should I do with 0 ?
what should I do with 1 ?
what should I do with 2 ?
what should I do with 3 ?
what should I do with 4 ?
what should I do with 5 ?
what should I do with 6 ?
what should I do with 7 ?
what should I do with 8 ?
what should I do with 9 ?


Na verdade, Python 2 tem uma função `lazy_range` chamada `xrange`. Para a nossa sorte, a função `range` do Python 3 já é preguiçosa.

Geradores são tão poderosos que você pode criar até uma sequência infinita de números:

In [12]:
def natural_numbers():
    """returns 1, 2, 3, ..."""
    n = 1
    while True:
        yield n
        n += 1

O Python 3 já oferece alguns iteradores preguiçosos para produzir loops eficientes. O iterador `count(start, step)` produz números a partir de um valor inicial `start` e com um passo `step` definido como parâmetro (o valor padrão do passo é `1`):

In [13]:
from random import random
from itertools import count

print('count(10) com parada aleatória:')
for i in count(10):
    if random() < 0.8:
        print(i, end=" ")
    else:
        break
        
print('\ncount(1,2) com parada aleatória:')        
for i in count(1, 2):
    if random() < 0.8:
        print(i, end=" ")
    else:
        break        

count(10) com parada aleatória:
10 
count(1,2) com parada aleatória:
1 3 

Outros iteradores preguiçosos interessantes são o `cycle` e o `repeat:

In [14]:
from itertools import repeat, cycle
print("cycle('ABCD') com parada aleatória:")
for letter in cycle('ABCD'):
    if random() < 0.95:
        print(letter, end=" ")
    else:
        break
        
print("\nrepeat(10,3):")
for i in repeat(10,3):
    print(i, end=" ")

cycle('ABCD') com parada aleatória:
A B C D A B C 
repeat(10,3):
10 10 10 

Você pode usar a função `islice` para fazer com que os iteradores acima gerem sequências finitas. 

In [15]:
from itertools import count, islice

print("Interrompe a geração de count(10) quando 2 números forem gerados:")
for number in islice(count(10), 2):
    print(number, end=" ")

print("\nInterrompe a geração count(1,3) quando 10 números forem gerados:")
for number in islice(count(1, 3), 10):
    print(number, end=" ")




Interrompe a geração de count(10) quando 2 números forem gerados:
10 11 
Interrompe a geração count(1,3) quando 10 números forem gerados:
1 4 7 10 13 16 19 22 25 28 

Os iteradores acima podem ser usado, por exemplo, em um programa que gera números primos grandes:

In [16]:
from itertools import count, islice
from math import sqrt;
        
def isPrime(n):
    if n < 2: return False
    for number in islice(count(2), int(sqrt(n)-1)):
        if not n%number:
            return False
    return True        
      
maiorPrimo = 1
for i in count(1):
    if(isPrime(i) and i > 500000):
        maiorPrimo = i

KeyboardInterrupt: 

Execute a célula anterior e a interrompa quando achar que deve. Depois execute a célula abaixo:

In [17]:
print(maiorPrimo)

15745039


**O outro lado de construir geradores preguiçosos é que você só pode iterar através deles uma única vez. Se precisar de iterar através deles várias vezes, você precisará recriar o gerador a cada vez ou usar uma lista.**

Uma segunda maneira de criar geradores é através do uso de formadores `for` agrupados em parênteses:

In [18]:
lazy_evens_below_20 = (i for i in range(20) if i % 2 == 0)
for n in lazy_evens_below_20:
    print(n, end=" ")

0 2 4 6 8 10 12 14 16 18 

Lembre-se também de que todo `dict` possui um método `items()` que retorna uma lista de seus pares de `(chave, valor)`. No Python 3, este método é preguiçoso.

## Aleatoriedade

À medida que aprendemos ciência dos dados, a gente precisará, com muita frequência, gerar números aleatórios. Isso pode ser feito com o seguinte módulo:

In [19]:
import random

four_uniform_randoms = [random.random() for _ in range(4)]
print(four_uniform_randoms)

[0.5245205292440019, 0.9149919135304775, 0.3538377534477207, 0.20229886768855243]


A função `random.random()` produz números uniformemente distribuídos entre 0 e 1. Usaremos muito essa função ao longo do curso.

O módulo `random`, na verdade, produz números pseudo-aleatórios, ou seja, eles dependem de um estado interno (ou semente) que você pode alterar usando `random.seed`. Isso serve para podermos reproduzir (ou repetir) os resultados gerados a partir de códigos que processam sequências de números aleatórios.

In [20]:
random.seed(10) # atribui 10 à semente
print('primeiro número: ', random.random())
random.seed(10) # atribui 10 à semente novamente
print('segundo número: ', random.random())

primeiro número:  0.5714025946899135
segundo número:  0.5714025946899135


Iremos usar `random.randrange` as vezes. Essa função recebe 1 ou 2 argumentos e retorna um elemento escolhido aleatoriamente entre o `range()` correspondente:

In [21]:
a = random.randrange(10) # range(10) = 0, 1, ..., 9
b = random.randrange(30, 60) # range(3, 6) = 30, 31, ..., 60
print(a,b)

6 45


Existem outros poucos métodos que nós acharemos conveniente usar, como o `random.shuffle`, que rearranja os elementos de uma lista de forma aleatória:

In [22]:
up_to_ten = list(range(10))
random.shuffle(up_to_ten)
print(up_to_ten)

[4, 5, 8, 1, 2, 6, 7, 3, 0, 9]


Se você precisa de um elemento aleatório de uma lista você pode usar `random.choice`:

In [23]:
my_best_friend = random.choice(["Alice", "Bob", "Charlie"])
print(my_best_friend)

Bob


Se você precisar escolher uma amostra aleatória de elementos sem reposição, você pode usar `random.sample`:

In [24]:
lottery_numbers = list(range(60))
winning_numbers = random.sample(lottery_numbers, 6)
print(winning_numbers)

[4, 15, 47, 23, 2, 26]


Para escolher uma amostra de $k$ elementos com reposição você pode fazer uma chamada para `random.choices` passando $k$ como parâmetro:

In [27]:
four_with_replacement = random.choices(list(range(3)), k=4)
print(four_with_replacement)

[2, 1, 1, 0]


Essa função também suporta pesos para cada elemento.

## Expressões Regulares

Expressões regulares fornecem uma maneira de pesquisar em textos. Elas são incrivelmente úteis, mas também bastante complicadas, tanto que há livros inteiros escritos sobre elas. Vamos explicar seus detalhes nas poucas vezes em que as encontrarmos. 

Aqui estão alguns exemplos de como usá-las em Python:

In [ ]:
import re

print([
    re.match("a", "gato"), # 'gato' não começa com 'a'
    re.search("c", "dog"), # 'dog' não tem 'c'
    re.split("[ab]", "carbs"), # usa 'a' e 'b' como separadores de "carbs"
    re.sub("[0-9]", "-", "R2D2") # substitui os dígitos por '-'
    ])


## Programação Orientada a Objetos

Como muitas linguagens, o Python permite que você defina classes que encapsulem dados e as funções que operam neles. Usaremos elas algumas vezes para tornar nosso código mais limpo e simples. É provavelmente mais simples explicá-las construindo um exemplo altamente anotado.

Imagine que não tivéssemos o objeto `set` interno do Python. Então poderíamos querer criar a nossa própria classe `Conjunto`.

Qual comportamento que a nossa classe `Conjunto` deveria ter? Dada uma instância de `Conjunto`, precisamos adicionar itens a ela, remover itens dela e verificar se ela contém um determinado valor. Vamos implementar todas essas funcionalidades como funções da classe, o que significa que vamos acessá-las com um ponto depois de um objeto `Conjunto`:

In [ ]:
# por convenção, daremos nomes no estilo PascalCase
class Conjunto:
    # essas são as funções da classe
    # todas devem ter como primeiro parâmetro o "self" (outra convenção),
    # que refere-se ao Set particular que está sendo usado

    def __init__(self, values=None):
        """Este é o construtor
        Ele é chamado quando você cria um novo Conjunto
        Você deve usá-lo da seguinte maneira:
        s1 = Conjunto() # conjunto vazio
        s2 = Conjunto([1,2,2,3]) # inicia com valores"""
        self.dict = {} # cada instância de Conjunto tem seu próprio dicionário 'dict'
        # o dict será usado para registrar se um valor já é membro do conjunto
        if values is not None:
            for value in values:
                self.add(value)

    def __repr__(self):
        """essa é a representação textual (string) do objeto Conjunto
        quando você o digitar no prompt do Python ou o enviar para str()"""
        return "Conjunto: " + str(self.dict.keys())

    # representaremos associação de um valor ao conjunto definindo-o como chave no self.dict do valor True
    def add(self, value):
        self.dict[value] = True

        
    # o valor está no conjunto se é uma chave no dicionário
    def contains(self, value):
        return value in self.dict

    def remove(self, value):
        del self.dict[value]

Podemos usar a classe `Conjunto` da seguinte maneira:

In [ ]:
s = Conjunto([1,2,3])
s.add(4)

print(s.contains(4))

s.remove(3)
print(s.contains(3))

## Ferramentas Funcionais

Às vezes precisamos de uma função complicada, que requer muitos parâmetros, mas sempre a usamos variando apenas um ou dois parâmetros. Assim, podemos criar novas funções, mais simples, a partir de funções existentes, mais complexas. Como um exemplo simples, imagine que temos uma função de dois parâmetros:

In [ ]:
def exp(base, power):
    return base ** power

e queremos usá-la para criar uma função de um parâmetro `dois_elevado_a`, cuja entrada é uma potência e cuja saída é o resultado de `exp(2, power)`.

Podemos, claro, fazer isso com `def`, mas às vezes isso pode ficar complicado:

In [ ]:
def dois_elevado_a(power):
    return exp(2, power)

Uma abordagem diferent é usar `functools.partial`:

In [ ]:
from functools import partial

dois_elevado_a = partial(exp, 2) # é agora uma função de um parâmetro
print(dois_elevado_a(3))

Você pode também usar `partial` para preencher argumentos se você especificar os seus nomes:

In [ ]:
quadrado_de = partial(exp, power=2)
print(quadrado_de(4))

Começa a ficar confuso se você carregar argumentos no meio da função, então vamos tentar evitar isso.

Também usaremos ocasionalmente `map`, `reduce` e `filter`, que fornecem alternativas funcionais para listas:

In [ ]:
def double(x):
    return 2 * x

xs = [1, 2, 3, 4]
twice_xs = [double(x) for x in xs] 
print('1: ', twice_xs)

twice_xs = list(map(double, xs)) # same as above
print('2: ', twice_xs)

list_doubler = partial(map, double) # função que duplica os itens de uma lista
twice_xs = list(list_doubler(xs))
print('3: ', twice_xs)

Você pode usar também `map` com funções de múltiplos argumentos se você prover múltiplas listas:

In [ ]:
def multiply(x, y): return x * y

products = list(map(multiply, [1, 2], [4, 5])) 

print(products)

De forma similar, `filter` faz o trabalho do `if` em listas:

In [ ]:
def is_even(x):
    """True if x is even, False if x is odd"""
    return x % 2 == 0

x_evens = [x for x in xs if is_even(x)] 
print('1: ', x_evens)

x_evens = list(filter(is_even, xs))
print('2: ', x_evens)

list_evener = partial(filter, is_even)
x_evens = list(list_evener(xs)) 
print('3: ', x_evens)

Por fim, `reduce` combina os dois primeiros elementos de uma lista, e depois o resultado disso com o terceiro da lista, e depois o resultado disso com o quarto, e assim por diante, produzindo um único resultado:

In [ ]:
from functools import reduce

x_product = reduce(multiply, xs)
print('1: ', x_product)

list_product = partial(reduce, multiply) # função que reduz uma lista
x_product = list_product(xs)
print('2: ', x_product)

*Observação importante*: `reduce` foi removido das funções internas do Python 3. É necessário usar, então, `functools.reduce` se você realmente precisar. No entanto, 99% das vezes um loop `for` explícito é mais legível.

## Enumerar

Muitas vezes você vai precisar iterar sobre uma lista e usar tanto o seus valores quanto os seus índices:

In [ ]:
def do_something(pos, word):
    if pos%10==0:
        print(word, end=" ")

lyric = """Rumours know that rebellion will break out
Bonnie Prince Charles is in the highlands to claim his
crown no doubt
He raised his Standart at Glenfinnen calling to our pride
The Jacobites are gathering I'll be at their side
Armed and ready stand
My rights I must defend
Steel is in my hand
The clan's are marching `gainst the law
Bagpipers play the tunes of war
Death or glory I will find
Rebellion on my mind
The town of Edinburgh fell soon in our hands
Defeated the English at the Battle of Prestopans"""

In [ ]:
words = lyric.split()

# não é pythônico
for i in range(len(words)):
    word = words[i]
    do_something(i, word)
    
print('\n')

# não é pythônico
i = 0
for word in words:
    do_something(i, word)
    i += 1

A solução pythônica é enumerar (ou `enumerate`), que produz tuplas `(index, element)`:

In [ ]:
for i, word in enumerate(words):
    do_something(i, word)

Similarmente, se quisermos somente os índices:

In [ ]:
from random import random

def do_stuff(idx):
    if random() < 0.5:
        print(words[idx], end=" ")

for i in range(len(words)): do_stuff(i) # não é pythônico

print("\n")

for i, _ in enumerate(words): do_stuff(i) # pythônico


Usaremos muito isso!

## zip e desempacotamento de argumentos

Frequentemente precisaremos empacotar (`zip`) duas ou mais listas juntas. A função `zip` transforma múltiplas listas em uma lista simples de tuplas dos elementos correspondentes:

In [ ]:
list1 = ['a', 'b', 'c']
list2 = [1, 2, 3]
list3 = list(zip(list1, list2))
print(list3)

Se as listas têm tamanhos diferentes, `zip` para assim que a primeira lista terminar.

Você pode também descompactar (*unzip*) uma lista usando um truque um tanto quanto estranho:

In [ ]:
letters, numbers = zip(*list3)
print(letters, numbers)

O asterisco executa o que chamamos de desempacotamento de argumentos (ou *argument unpacking*), que usa os elementos de `list3` como argumentos individuais para o `zip`. É a mesma coisa que fazer:

In [ ]:
letters, numbers = zip(('a', 1), ('b', 2), ('c', 3))
print(letters, numbers)

Você pode usar desempacotamento de argumentos com qualquer função:

In [ ]:
def add(a, b): return a + b

print(add(1, 2))

# add([1, 2]) #TypeError

print(add(*[1, 2]))


É raro acharmos isso útil, mas quando o fazemos é um truque legal.

## args e kwargs

Digamos que queremos criar uma função de alta ordem que tome como entrada alguma função `f` e retorne uma nova função que, para qualquer entrada, retorne duas vezes o valor de `f`:

In [ ]:
def doubler(f):
    def g(x):
        return 2 * f(x)
    return g

Isso funciona em alguns casos:

In [ ]:
def soma1(x):
    return x + 1

g = doubler(soma1)
print(g(3)) # 8 (== ( 3 + 1) * 2)
print(g(-1)) # 0 (== (-1 + 1) * 2)

Mas não funciona com funções que levam mais de um único argumento:

In [ ]:
def soma(x, y):
    return x + y

g = doubler(soma)
print(g(1, 2))

O que precisamos é uma maneira de especificar uma função que aceita argumentos arbitrários. Podemos fazer isso com argumento de desempacotamento e um pouco de mágica:

In [ ]:
def magic(*args, **kwargs):
    print("unnamed args:", args)
    print("keyword args:", kwargs)

magic(1, 2, key="word", key2="word2")

Ou seja, quando definimos uma função como essa, `args` é uma tupla de seus argumentos sem nome e `kwargs` é um dict de seus argumentos nomeados. Ele funciona de outra maneira também, se você quiser usar uma lista (ou tupla) e dicionário para fornecer argumentos para uma função:

In [5]:
def other_way_magic(x=0, y=0, w=0, z=0):
    return x + y + z + w

x_y_list = [1, 2]
z_dict = { "z" : 3 }
print(other_way_magic(*x_y_list, **z_dict))

6




O comando `**z_dict` descompacta o dicionário que contem a chave `z` e que é o nome do argumento da função other_way_magic. Se o dicionário tiver chaves diferentes dos argumentos da função, então o nosso truque não funcionará.

Você poderia fazer todo tipo de truques estranhos com isso; Vamos usá-lo apenas para produzir funções de ordem superior cujas entradas podem aceitar argumentos arbitrários:

In [ ]:
def doubler_correct(f):
    """works no matter what kind of inputs f expects"""
    def g(*args, **kwargs):
        """whatever arguments g is supplied, pass them through to f"""
        return 2 * f(*args, **kwargs)
    return g

g = doubler_correct(soma)
print(g(1, 2))

## Para explorar

Não há escassez de tutoriais em Python no mundo. O [oficial](https://docs.python.org/3/tutorial/) não é um mau lugar para começar.

O [tutorial oficial do IPython](http://ipython.org/ipython-doc/2/interactive/tutorial.html) não é tão bom. Pode ser melhor assistir a [vídeos e apresentações](http://ipython.org/videos.html). Como alternativa, há o capítulo [*Python for Data Analysis*](http://shop.oreilly.com/product/0636920023784.do) do Wes McKinney (O’Reilly) que é muito bom!